In [2]:
from torch import nn
import torch
import torch.nn.functional as F


class VisionTransformer(nn.Module):

    def __init__(self, in_channels: int, out_channels: int, hidden_size: int, patch_size: int, num_layers: int):
        super().__init__()
        self.positional_embeds = nn.Embedding(100000, hidden_size)
        self.register_buffer("positional_ids", torch.arange(100000).unsqueeze(0))
        self.patch_conv = nn.Conv2d(in_channels, hidden_size, patch_size, stride=patch_size)
        self.patch_deconv = nn.ConvTranspose2d(hidden_size, out_channels, patch_size, stride=patch_size)
        self.model = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
            hidden_size,
            hidden_size // 64,
            hidden_size * 8 // 3,
            activation=F.silu,
            batch_first=True,
            norm_first=True,
            ),
            num_layers,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        hidden = self.patch_conv(x)
        n_seq = hidden.shape[2] * hidden.shape[3]
        pos_embeds = self.positional_embeds(self.positional_ids[:, :n_seq]).expand(hidden.shape[0], -1, -1)
        output = self.model(hidden.flatten(2).transpose(1, 2) + pos_embeds).transpose(-1, -2).view_as(hidden)
        output = self.patch_deconv(output)
        return output

In [1]:
from torch.utils.data import Dataset
import numpy as np
import torch
import os


class EMGDataset(Dataset):

    def __init__(self, root_dir):
        super(EMGDataset, self).__init__()
        self.emg = np.load(os.path.join(root_dir, 'emg_train.npy'))
        self.force = np.load(os.path.join(root_dir, 'force_train.npy'))
        self.force_class = np.load(os.path.join(root_dir, 'force_class_train.npy'))

    def __len__(self):
        return len(self.emg)

    def __getitem__(self, idx):
        emg = self.emg[idx]
        force = self.force[idx]
        force_class = self.force_class[idx]
        return torch.from_numpy(emg), torch.from_numpy(force), torch.from_numpy(force_class)


In [8]:
root_dir = os.path.join(os.getcwd(), 'Dataset')
emg = np.load(os.path.join(root_dir, 'emg_train.npy'))
x = emg[:64]
x = torch.from_numpy(x)
x.shape

torch.Size([64, 8, 1248])

In [5]:
root_dir = os.path.join(os.getcwd(), 'Dataset')
force_class = np.load(os.path.join(root_dir, 'force_class_train.npy'))
print('f_c',force_class[:64].shape)
force = np.load(os.path.join(root_dir, 'force_train.npy'))
force[:64].shape

f_c (64, 5, 32)


(64, 5, 32)

In [9]:
hann_window = torch.hann_window(256)
x = x.view(64*8, x.size(2))
y = torch.stft(x, n_fft=256, 
                hop_length=32, 
                win_length=256, 
                window=hann_window,
                center=False, 
                pad_mode='reflect', 
                normalized=False, 
                onesided=True, 
                return_complex=True).abs()
print(x.shape, y.shape)

torch.Size([512, 1248]) torch.Size([512, 129, 32])


In [28]:
x = torch.rand((512, 129, 32))
x = x.view(64, 8, x.size(1), x.size(2))
x = x.transpose(2, 3)[..., :64]
x.shape

torch.Size([64, 8, 32, 64])

In [11]:
x = torch.rand((64, 32, 32, 1))


import torch.nn.functional as F
import torch.nn as nn
y = x.view(x.size(0), x.size(1), x.size(2))
y1 = y.transpose(1, 2)
n = nn.Linear(32, 2 * 5)
# x = self.linear(x)
# x = x.transpose(1, 2)
# x = x.view(x.size(0), self.num_force_levels, self.num_forces, x.size(2))
y2 = n(y1)
y3 = y2.transpose(1,2)
y4 = y3.view(y3.size(0), 2, 5, y3.size(2))
y4.shape

root_dir = os.path.join(os.getcwd(), 'Dataset')
force_class = np.load(os.path.join(root_dir, 'force_class_train.npy'))
print('f_c',force_class[:64].shape)
force = np.load(os.path.join(root_dir, 'force_train.npy'))
force[:64].shape
print(y4.shape)
print(torch.tensor(force_class[:64]).shape)
F.cross_entropy(y4, torch.tensor(force_class[:64]))

f_c (64, 5, 32)
torch.Size([64, 2, 5, 32])
torch.Size([64, 5, 32])


tensor(0.7209, grad_fn=<NllLoss2DBackward0>)

In [23]:
print(force_class[128:129])

[[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]]


In [49]:
a = torch.tensor(np.array([[1.0, 19999999999.0]]))
b = torch.tensor(np.array([1]).astype(np.longlong))
print(a.shape, b.shape)
F.cross_entropy(a, b)

torch.Size([1, 2]) torch.Size([1])


tensor(0., dtype=torch.float64)